In [5]:
from GABRIEL.combined_assistant import CombinedAssistant
import os

api_key = os.getenv('OPENAI_API_KEY')

In [7]:
combined_assistant = CombinedAssistant(api_key)

In [8]:
texts = ['This is a funny story' , "This is a sad story","John broke his leg"]
attributes = ['happy, sad, angry, surprised, grateful']

In [ ]:
combined_assistant.simple_evaluation_pipeline(search_axis_1 = texts, attributes = attributes)